### 기상청 날씨 조회
- 참조:https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36&tabNo=2

In [1]:
# 필요한 모듈 연결하기

from datetime import datetime, timedelta
import pandas as pd
import requests
import json

### 1. 조회를 위한 시작일/종료일 입력
[조건]
1. 시작일/종료일 입력받기(예: 2022-01-01)
2. 입력값이 10자리가 아닌 경우 다시 입력 받기
3. "="를 뺀 나머지 입력 데이터가 모두 숫자인지 아닌지 확인
4. 종료일은 시작일 이전의 데이터 입력 불가
5. 종료일이 오늘 날짜라면 -1일 진행

In [2]:
# 시작일 입력 함수
def startDate():
    # while문을 이용해 조건이 만족될때까지 계속 실행
    while True:
        # 데이터 입력
        sDate = input("조회 시작일 입력(예:2022-01-01)")
        
        if len(sDate) != 10:
            print('"2022-01-01"와 같이 "-"를 포함해 10자리로 입력하세요.')
            continue
        
        # "="를 뺀 나머지 입력 데이터가 모두 숫자인지 아닌지 확인
        # 문자 포함시 다시 입력받기
        try:
            sdt = int(sDate.replace("-", ""))
            break
        except:
            print("년, 월, 일 자리에 문자가 포함되어 있습니다.\n다시 입력해주세요.")
            continue
            
    return sDate



In [3]:
sdt = startDate()
print(sdt)

조회 시작일 입력(예:2022-01-01)2022-01-31
2022-01-31


### 2. 종료일 입력
- 시작일과 종료일은 다른 값으로 입력
- 종료일은 시작일 이전의 데이터 입력 불가
- 종료일이 오늘 날짜라면 -1일 진행

In [18]:
def endDate():
    # while문을 이용해 조건이 만족될때까지 계속 실행
    
#     sDate = startDate()
    while True:
        # 데이터 입력
        eDate = input("종료일 입력(예:2022-01-01)")
        
        if len(eDate) != 10:
            print('"2022-01-01"와 같이 "-"를 포함해 10자리로 입력하세요.')
            continue
        
        # "="를 뺀 나머지 입력 데이터가 모두 숫자인지 아닌지 확인
        # 문자 포함시 다시 입력받기
        # 입력한 종료일이 오늘 날짜인지 확인 => 오늘날짜-1 한 날로 세팅
        # 입력한 종료일이 시작일보다 크거나 같은 값인지 확인 => 재입력
        try:
            edt = int(eDate.replace("-", ""))
            today = datetime.today()  # 현재 시스템 날짜 가져오기
            newDate = today.strftime('%Y%m%d')# 날짜형 => 문자형으로 변경

                # 입력데이터가 현재일보다 크거나 같은 데이터인지 확인
            if edt >= int(newDate):
                # 크거나 같다면 실행 전날로 종료일 변경
                eDate = (datetime.today()-timedelta(days = 1)).strftime("%Y-%m-%d")
                print('데이터는 당일 이전 자료까지만 제공합니다.',eDate)

            if int(sDate.replace("-","")) > edt:
                print("종료일이 시작일보다 이전 날짜입니다.\n다시 입력해주세요.")
                continue

            break
        except:
            print("년, 월, 일 자리에 문자가 포함되어 있습니다.\n다시 입력해주세요.")
            continue
            
    return sDate, eDate



In [12]:
dt = datetime.today().strftime("%Y%m%d")
print(dt, type(dt))
newdt = datetime.strptime(dt[2:], "%y%m%d")
print(newdt, type(newdt))
print(datetime.today()-timedelta(days = 1))


20220502 <class 'str'>
2022-05-02 00:00:00 <class 'datetime.datetime'>
2022-05-01 20:52:21.152788


### 3. 측정 지점 코드 확인
- 측정 지점의 이름을 입력하면 해당 지점 출력

In [13]:
def getRegCode(inRegName="서울"):
    df_regCode = pd.read_csv("기상청_지역코드.csv", encoding = "cp949")
    regCode = df_regCode[df_regCode['지점명'] == inRegName]  # 원하는 지점명 검색
#     display(regCode)
    
    if len(regCode) == 1:  # 지점명 검색 여부 확인
        return (int(regCode['지점'].values))  # 검색 되면 코드 전달
    else:
        # 검색 자료가 없으면 사용자가 강제 에러 발생
        raise Exception(f"해당 지역명은 없습니다.\n지역명 확인 후 입력하세요.\n\n")

### 4. 날씨데이터 조회 url 만들기
- 시작일/종료일을 기준으로 지점명에 해당하는 날씨정보 조회 url 생성

In [14]:
# url 생성
def getWturl(sDate, eDate, pointID, numRows = 10):
    Key = "rx1J4yPTJLMypgEsmw%2BuyunTeZzwwUbRPrwPUkkYNxyUDpxomPQu1Hjz8vXE%2BzLfrzvFQdKPXqp%2FvonN%2Bc77iA%3D%3D"
    url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=" + Key
    url = url + "&pageNo=1&dataCd=ASOS&dateCd=DAY&numOfRows=" + str(numRows)
    url = url + "&dataType=JSON"
    url = url + "&startDt=" + str(sDate)
    url = url + "&endDt=" + str(eDate)
    url = url + "&stnIds=" + str(pointID)

    return url

### 5. url을 이용해 실제 데이터 조회 및 DataFrame으로 저장
- 1차 조회 후 전체 데이터 개수 확인
- 전체 데이터 개수를 이용한 1페이지에 조회 데이터 전체 가져오기
- 가져온 데이터에서 필요한 데이터만 DataFrame으로 최종 출력하기

In [15]:
def getJsonData(url):
    soup_json = requests.get(url)
    if soup_json.status_code != 200:
        exit(f"데이터를 받지 못했습니다. 에러코드:{soup_json.status_code}")
    json_obj = json.loads(soup_json.text) # json 코드 읽어오기/ 딕셔너리 구조


    return json_obj

In [16]:
def getDataFrame(sDate, eDate, pointID):
    url = getWturl(sDate.replace("-",""), eDate.replace("-",""), pointID)  # 1차 url 생성
    json_obj = getJsonData(url) # 1차 데이터 추출

    numRows = json_obj["response"]["body"]["totalCount"]  # 조회데이터 건수

    url = getWturl(sDate.replace("-",""), eDate.replace("-",""), pointID, numRows)  # url 생성
    json_obj = getJsonData(url)  # 최종 데이터 추출
    items = json_obj["response"]["body"]["items"]["item"] # 필요한 item 값만 가져오기
    df1 = pd.DataFrame(items) # items(딕셔너리 구조)를 DataFrame으로 변경
    
    df1 = df1[['stnNm', "tm", "avgTa", "minTa", "maxTa", "sumRn"]] # 원하는 열만 추출
    
    return df1

In [19]:
sDate, eDate = endDate() # 시작일/종료일 사용자 입력 입력
# sDate = datetime.today() + relativedelta(years = 1, days = -1)
pointName = input("지점명 입력:") # 지점명 입력
pointID = getRegCode(pointName) # 지점명에 의한 지점코드 호출

df1 = getDataFrame(sDate, eDate, pointID)
df1.info()

종료일 입력(예:2022-01-01)2022-03-01
년, 월, 일 자리에 문자가 포함되어 있습니다.
다시 입력해주세요.
종료일 입력(예:2022-01-01)2022-03-01
년, 월, 일 자리에 문자가 포함되어 있습니다.
다시 입력해주세요.
종료일 입력(예:2022-01-01)2022003001
데이터는 당일 이전 자료까지만 제공합니다. 2022-05-01
년, 월, 일 자리에 문자가 포함되어 있습니다.
다시 입력해주세요.
종료일 입력(예:2022-01-01)2022-01-01
년, 월, 일 자리에 문자가 포함되어 있습니다.
다시 입력해주세요.
종료일 입력(예:2022-01-01)2022-03-23
년, 월, 일 자리에 문자가 포함되어 있습니다.
다시 입력해주세요.


KeyboardInterrupt: Interrupted by user

### 6. 시작일을 입력하면 해당일부터 오늘까지 데이터 전체를 조회하기


- 기준일 입력 => 해당 일을 기준으로 1월부터 12월까지 데이터 조회

In [20]:
sDate = "2000-03-01"

sdt = pd.date_range("2000-03-23", datetime.today(), freq = "MS")
print(sdt)


sdt = pd.date_range("2000-03-23", datetime.today(), freq = "M")
print(sdt)

sdt = pd.date_range("2000-03-23", datetime.today(), freq = "D")
print(sdt)

DatetimeIndex(['2000-04-01', '2000-05-01', '2000-06-01', '2000-07-01',
               '2000-08-01', '2000-09-01', '2000-10-01', '2000-11-01',
               '2000-12-01', '2001-01-01',
               ...
               '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01',
               '2021-12-01', '2022-01-01', '2022-02-01', '2022-03-01',
               '2022-04-01', '2022-05-01'],
              dtype='datetime64[ns]', length=266, freq='MS')
DatetimeIndex(['2000-03-31', '2000-04-30', '2000-05-31', '2000-06-30',
               '2000-07-31', '2000-08-31', '2000-09-30', '2000-10-31',
               '2000-11-30', '2000-12-31',
               ...
               '2021-07-31', '2021-08-31', '2021-09-30', '2021-10-31',
               '2021-11-30', '2021-12-31', '2022-01-31', '2022-02-28',
               '2022-03-31', '2022-04-30'],
              dtype='datetime64[ns]', length=266, freq='M')
DatetimeIndex(['2000-03-23', '2000-03-24', '2000-03-25', '2000-03-26',
               '2000-03-27',

In [22]:


pointName = input("지점명 입력:") # 지점명 입력
pointID = getRegCode(pointName) # 지점명에 의한 지점코드 호출

df1 = getDataFrame(sDate, eDate, pointID)
df1.info()

지점명 입력:서울


FileNotFoundError: [Errno 2] No such file or directory: '/Users/wondongchan/Desktop/multi/기상청_지역코드.csv'

[미션] 위 코드의 결과를 이용해 날짜를 추출해 하나의 DF로 출력

In [ ]:
sDate = "2000-03-30"

date_list = []
sdt = int(sDate[:4])   # 입력 데이터에서 4개 출력
edt = datetime.today().year   # 오늘 날짜에서 년도만 빼오기
print(sdt, edt)
cnt = 0

for i in range(sdt, edt+1):
    if cnt == 0:
        date_list.append([sDate, str(i)+"-12-31"])
        cnt = 1
    else:
        date_list.append([str(i)+"-01-01", str(i)+"-12-31"])
    
print(date_list)
        

In [ ]:
pointName = input("지점명 입력:") # 지점명 입력
df = pd.DataFrame()

for i in date_list:
    pointID = getRegCode(pointName) # 지점명에 의한 지점코드 호출
    if int(dt[1].replace("-","")) >= int(datetime.today().strftime("%Y-%m-%d")):
        edt = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")
        df1 = getDataFrame(dt[0], edt, pointID)
    else:
        df1 = getDataFrame(dt[0], dt[1], pointID)
    df = pd.concat([df,df1], ignore_index=True)

df.info()

In [ ]:
from dateutil.relativedelta import relativedelta

print(datetime.today() + relativedelta(months = 1)) # 월단위 증감
print(datetime.today() + relativedelta(years = 1)) # 년단위 증감

